In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.stats import norm
from mpl_toolkits.mplot3d import Axes3D


In [ ]:
def simulation(stock_ticker,start_date,end_date,trial_count,daysofsim):
    trading_days= 252
    data = yf.download(stock_ticker, start_date ,end_date)
    time_elapsed = (data.index[-1]- data.index[0]).days
    total_growth = (data['Adj Close'][-1] / data['Adj Close'][1])
    years =  time_elapsed/365.0
    cagr= total_growth ** (1/years) - 1
    std_dev= data['Adj Close'].pct_change().std()
    std_dev = std_dev * math.sqrt(trading_days)
    closing_prices = []
    for i in range(trial_count):
        daily_return_percentages = np.random.normal(cagr/trading_days , std_dev / math.sqrt(trading_days), daysofsim)+1
        price_series = [data['Adj Close'][-1]]

        for j in daily_return_percentages:
            price_series.append(price_series[-1]*j)

        closing_prices.append(price_series[-1])

        plt.plot(price_series)


    plt.show()
    plt.hist(closing_prices , bins = 40)
    plt.show()
    mean_end_price = round(np.mean(closing_prices),2)
    #print("Expected Price: ", mean_end_price)
    return closing_prices, std_dev, data





    

In [ ]:
closing_prices , volatility, data = simulation('SPY','2021-01-01','2024-01-01',1000,120)

In [ ]:
def percent_itm(strike_price, closing_prices, trial_count):
    comparison_results = [num - strike_price for num in closing_prices]
    positive_results = [num for num in comparison_results if num > 0]
    negative_results = [num for num in comparison_results if num <= 0]
    profitability = len(positive_results) / trial_count
    #print(profitability)
    plt.hist(positive_results, bins=40, color='blue', label='Positive Results')
    plt.hist(negative_results, bins=40, color='red', label='Negative Results')
    plt.legend()
    plt.show()
    return profitability, comparison_results, trial_count

    

In [ ]:
profitability, comparison_results, trial_count = percent_itm(370,closing_prices, 1000)


In [ ]:
def all_path_exepected_returns(profitability,comparison_results, trial_count):
    expected_profit = 0
    for n in comparison_results:
        expected_profit += (n*profitability)*(1/trial_count) 
    return expected_profit
    
    
    
    

In [ ]:
all_path_expected_profit = all_path_exepected_returns(profitability,comparison_results, trial_count)

In [ ]:
def black_scholes(S0, x, r, T, volatility ):
    d1 = (np.log(S0/x) + (r + ((volatility)**2)/2)*T)/(volatility*math.sqrt(T))

    d2 = (np.log(S0/x) + (r - ((volatility)**2)/2)*T)/(volatility*math.sqrt(T))

    cost = S0*norm.cdf(d1) - x*((math.e)**(-1*r*T))*norm.cdf(d2)

    return cost    

In [ ]:
def optimization(stock_ticker , start_date , end_date ,trial_count, daysofsim , strike_price):
    closing_prices , volatility, data = simulation(stock_ticker,start_date,end_date,trial_count,daysofsim)
    profitability, comparison_results, trial_count = percent_itm(strike_price,closing_prices, trial_count)
    all_path_expected_profit = all_path_exepected_returns(profitability,comparison_results, trial_count)
    
    T = daysofsim/252
    
    price_of_option = black_scholes(data['Open'][-1],strike_price, 0.0428, T , volatility)

    net_profit = all_path_expected_profit-price_of_option
    #print("Net Profit:", net_profit)  # Debugging statement


    #print(net_profit)
    return net_profit
    

In [ ]:
optimization('SPY','2021-01-01','2024-01-01',1000,120,370)

In [ ]:
def function(stock_ticker , start_date , end_date ,trial_count, minprice , maxprice , minday, maxday):
    prices = list(range(minprice,maxprice+1))
    results  = {}
    dates = list(range(minday,maxday+1))

    for sprice in prices:
        for day in dates:
            net_profit = optimization(stock_ticker , start_date , end_date ,trial_count, day , sprice)
            results[(sprice,day)] = net_profit
            print(f"Strike Price: {sprice}, Contract Length: {day}, Net Profit: {net_profit}")
            return results

            



In [ ]:
function('SPY','2021-01-01','2024-01-01',100,350,360,1,10)

In [ ]:
def main_function(stock_ticker, start_date, end_date, trial_count, minprice, maxprice, minday, maxday):
    prices = list(range(minprice, maxprice + 1))
    days = list(range(minday, maxday + 1))
    results = {}

    for sprice in prices:
        for day in days:
            net_profit = optimization(stock_ticker, start_date, end_date, trial_count, day, sprice)
            results[(sprice, day)] = net_profit
            print(f"Strike Price: {sprice}, Contract Length: {day}, Net Profit: {net_profit}")
    
    return results



In [ ]:
results = main_function('SPY','2021-01-01','2024-01-01',100,425,500,1,100)

In [ ]:
def find_max_min(results):
    max_key = max(results, key=results.get)
    min_key = min(results, key=results.get)
    max_value = results[max_key]
    min_value = results[min_key]
    return max_key, max_value, min_key, min_value

In [ ]:
def plot_results(results):
    strike_prices = sorted(set(k[0] for k in results.keys()))
    contract_lengths = sorted(set(k[1] for k in results.keys()))
    Z = np.array([results[(sp, cl)] for sp in strike_prices for cl in contract_lengths])
    Z = Z.reshape(len(strike_prices), len(contract_lengths))

    X, Y = np.meshgrid(contract_lengths, strike_prices)
    
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(X, Y, Z, cmap='viridis')

    ax.set_xlabel('Contract Length')
    ax.set_ylabel('Strike Price')
    ax.set_zlabel('Net Profit')

    # Find and highlight max and min points
    max_key, max_value, min_key, min_value = find_max_min(results)
    max_x, max_y = max_key
    min_x, min_y = min_key

    ax.scatter(min_y, min_x, min_value, color='r', s=100, label='Min')
    ax.scatter(max_y, max_x, max_value, color='r', s=100, label='Max')

    ax.legend()

    plt.show()

    # Print max and min values
    print(f"Max Net Profit: {max_value} at Strike Price: {max_key[0]}, Contract Length: {max_key[1]}")
    print(f"Min Net Profit: {min_value} at Strike Price: {min_key[0]}, Contract Length: {min_key[1]}")



In [ ]:
plot_results(results)